In [2]:
import pickle 
import pandas as pd 
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
import torch.optim as optim

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

25104


In [5]:
file = open(r'C:\Users\Nick\Documents\GitHub\grvmodel\finalnew.txt', 'rb')
data = pickle.load(file)
#seperate out classes from inputs
classes, inputs = zip(*data)

#removing nan from inputs and convert to float
inputs_df = pd.DataFrame(inputs)
inputs_df.fillna(value=-1,inplace = True)
inputs = inputs_df.values.tolist()
inputs = [[float(i) for i in j] for j in inputs]

In [6]:
REBUILD_DATA = False

class GRV():
    #class to store training data
    
    #reading data from pickle
    file = open(r'C:\Users\Nick\Documents\GitHub\grvmodel\finalnew.txt', 'rb')
    data = pickle.load(file)
    #seperate out classes from inputs
    classes, inputs = zip(*data)
    
    #removing nan from inputs and convert to float
    inputs_df = pd.DataFrame(inputs)
    inputs_df.fillna(value=-1,inplace = True)
    inputs = inputs_df.values.tolist()
    inputs = [[float(i) for i in j] for j in inputs]
    
    
    
    #data
    training_data = []

    def make_training_data(self):
        excluded = 0
        for i in range(len(inputs)):
            if len(classes[i]) == 8:
                self.training_data.append([np.array(inputs[i]),np.array(classes[i])])
            else:
                excluded += 1
        np.save('training_data.npy', self.training_data)
        print("excluded = ", excluded)
        
if REBUILD_DATA:
    grv = GRV()
    grv.make_training_data()
    
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(409, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 8)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x #nn.Softmax(x, dim=1)
        
        
net = Net().to(device)
print(net)    
#define tensors

X = torch.Tensor([i[0] for i in training_data])
Y = torch.Tensor([i[1] for i in training_data])
#Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)
    
Y_w = torch.tensor([i for i in Y_w])
    
VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)
train_X = X[:-val_size]
train_y = Y_w[:-val_size]

test_X = X[-val_size:]
test_y = Y_w[-val_size:]
print(len(train_X), len(test_X))

test_X = test_X.to(device)
test_y = test_y.to(device)
    
def train(net):
    BATCH_SIZE = 100
    EPOCHS = 1000
    loss_function = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.05)

    for epoch in tqdm.tqdm(range(EPOCHS)):
        for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE]
            batch_y = train_y[i:i+BATCH_SIZE]

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            net.zero_grad()
            
            m = nn.LogSoftmax(dim = 1)
            outputs = m(net(batch_X))
            loss = loss_function(outputs, batch_y.float())
            loss.backward()
            optimizer.step()    # Does the update



        #print(f"Epoch: {epoch}. Loss: {loss}")
        #test(net)
        if (epoch%100 == 0):
            print(f"Epoch: {epoch}. Loss: {loss}")
            test(net)
            
def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm.tqdm(range(len(test_X))):
            #print("test_y[i] \n", test_y[i], "\noutput y =", net(test_X[i]))
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i]).to(device)  # returns a list, 
            #print(net_out)
            predicted_class = torch.argmax(net_out)
            #print("predicted_class = ", predicted_class,"\nreal class = ", real_class)
            
            #print(predicted_class)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))

Net(
  (fc1): Linear(in_features=409, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=8, bias=True)
)
2510
22594 2510


In [7]:
X = torch.Tensor([i[0] for i in training_data])
Y = torch.Tensor([i[1] for i in training_data])

In [8]:
#Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)
Y_w = torch.tensor([i for i in Y_w])
Y_w[1:10]

tensor([[0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.]], dtype=torch.float64)

In [9]:
#indexed win only
Y_wi = []
for i in Y:
    #print(torch.argmin(i))
    Y_wi.append(torch.argmin(i))
Y_w = torch.tensor(Y_wi)
print(Y_w)

tensor([3, 4, 1,  ..., 0, 4, 5])


In [ ]:
#Generates places
Y_p = []
for i in Y:
    place = [x*x for x in i]
    Y_p.append(place)
    
Y_p[1]

In [20]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(409, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 8)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
        
net = Net().to(device)
print(net)

Net(
  (fc1): Linear(in_features=409, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=8, bias=True)
)


In [21]:
#validating model is new
for (i,j) in net.named_parameters():
    print(i,j)

fc1.weight Parameter containing:
tensor([[ 0.0003, -0.0287,  0.0260,  ..., -0.0343, -0.0237,  0.0453],
        [ 0.0209,  0.0221,  0.0139,  ..., -0.0159, -0.0340,  0.0090],
        [ 0.0118, -0.0334, -0.0321,  ...,  0.0333,  0.0097,  0.0253],
        ...,
        [-0.0230,  0.0069,  0.0276,  ...,  0.0223,  0.0233,  0.0301],
        [ 0.0048, -0.0066,  0.0234,  ..., -0.0289, -0.0260,  0.0068],
        [ 0.0278, -0.0488,  0.0087,  ..., -0.0198,  0.0134,  0.0193]],
       device='cuda:0', requires_grad=True)
fc1.bias Parameter containing:
tensor([ 0.0071,  0.0395,  0.0162,  0.0059, -0.0032,  0.0381,  0.0053,  0.0232,
        -0.0292,  0.0161, -0.0021,  0.0048, -0.0166,  0.0407, -0.0308,  0.0355,
        -0.0285, -0.0333,  0.0283,  0.0160, -0.0229, -0.0045,  0.0448,  0.0307,
         0.0433, -0.0219, -0.0026, -0.0287,  0.0424, -0.0359,  0.0212, -0.0476,
        -0.0193, -0.0339, -0.0136,  0.0336, -0.0298, -0.0115,  0.0385, -0.0333,
        -0.0348,  0.0341, -0.0251,  0.0473,  0.0294,  0.02

In [19]:
VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)
train_X = X[:-val_size]
train_y = Y_w[:-val_size]

test_X = X[-val_size:]
test_y = Y_w[-val_size:]
print(len(train_X), len(test_X))

test_X = test_X.to(device)
test_y = test_y.to(device)

2510
22594 2510


In [134]:
#DONT RUN


VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)
train_X = X[:-val_size]
train_y = Y[:-val_size]

test_X = X[-val_size:]
test_y = Y[-val_size:]
print(len(train_X), len(test_X))



2510
22594 2510


In [ ]:
net = Net().to(device)

In [18]:
#utalized Win only class, not usefull for ranking placement

def train(net):
    BATCH_SIZE = 100
    EPOCHS = 100
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.0001)

    for epoch in tqdm.tqdm(range(EPOCHS)):
        for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE]
            batch_y = train_y[i:i+BATCH_SIZE]

            #print(batch_X, batch_y)

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            net.zero_grad()
            optimizer.zero_grad()
            m = nn.LogSoftmax(dim = 1)
            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y.long())
            loss.backward()
            optimizer.step()    # Does the update



        #print(f"Epoch: {epoch}. Loss: {loss}")
        #test(net)
        if (epoch%10 == 0):
            print(f"Epoch: {epoch}. Loss: {loss}")
            test(net)

In [22]:
train(net)

100%|██████████| 100/100 [00:37<00:00,  2.67it/s]


In [ ]:
def test(net):
    correct = 0
    total = 0
    realclasslist = []
    predictedClassList = []
    with torch.no_grad():
        for i in range(len(test_X)):
            #print("test_y[i] \n", test_y[i], "\noutput y =", net(test_X[i]))
            real_class = torch.argmax(test_y[i]).to(device)
            realclasslist.append(test_y[i])
            net_out = net(test_X[i]).to(device)  # returns a list, 
            predictedClassList.append(net_out.tolist())
            #print(net_out)
            predicted_class = torch.argmax(net_out)
            #print("predicted_class = ", predicted_class,"\nreal class = ", real_class)
            
            #print(predicted_class)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))
    return (predictedClassList, realclasslist)

In [16]:
#Test for indexed win only


def test(net):
    correct = 0
    total = 0
    realclasslist = []
    predictedClassList = []
    with torch.no_grad():
        for i in range(len(test_X)):
            #print("test_y[i] \n", test_y[i], "\noutput y =", net(test_X[i]))
            real_class = test_y[i]
            #print(real_class)
            realclasslist.append(test_y[i])
            net_out = net(test_X[i]).to(device)  # returns a list, 
            predictedClassList.append(net_out.tolist())
            #print(net_out)
            predicted_class = torch.argmax(net_out)
            #print("predicted_class = ", predicted_class,"\nreal class = ", real_class)
            
            #print(predicted_class)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))
    return (predictedClassList, realclasslist)

In [17]:
results = test(net)

Accuracy:  0.139


In [132]:
winners = [i.item() for i in Y_wi]
from collections import Counter
  
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]
    
List = winners
print(most_frequent(List))

4


In [ ]:
#TESTing
m = nn.LogSoftmax(dim = 1)

In [ ]:
for i in range(1000):
    print(torch.argmax(net(test_X)[i]))

In [99]:
df = pd.DataFrame({'Number' : Y_wi})